In [1]:
import numpy as np
import tensorflow as tf
import cv2 as cv
from pathlib import Path
from send2trash import send2trash
from google_images_download import google_images_download
import os

x=os.getcwd()
# Read the graph.
with tf.io.gfile.GFile('frozen_inference_graph.pb', 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

In [4]:
def detect_persons_and_save_crops(asana, graph_def, folder):
    with tf.compat.v1.Session() as sess:
        # Restore session
        sess.graph.as_default()
        tf.import_graph_def(graph_def, name='')

        files = list(Path(folder).glob('*.jpg'))
        files.extend(list(Path(folder).glob('*.png')))

        for fid,fname in enumerate(files): 
            # Read and preprocess an image.
            img = cv.imread(str(fname))
            if(img is not None):
                rows = img.shape[0]
                cols = img.shape[1]
                inp = cv.resize(img, (400, 400))
                inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

                # Run the model
                out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                                sess.graph.get_tensor_by_name('detection_scores:0'),
                                sess.graph.get_tensor_by_name('detection_boxes:0'),
                                sess.graph.get_tensor_by_name('detection_classes:0')],
                               feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

                # Visualize detected bounding boxes.
                num_detections = int(out[0][0])
                #print(fname, num_detections)
                for i in range(num_detections):
                    classId = int(out[3][0][i])
                    if(classId==1):
                        score = float(out[1][0][i])
                        bbox = [float(v) for v in out[2][0][i]]
                        if score > 0.3:
                            x = int(bbox[1] * cols)
                            y = int(bbox[0] * rows)
                            right = int(bbox[3] * cols)
                            bottom = int(bbox[2] * rows)
                            cv.imwrite(folder+str(fid)+'-'+str(i)+'.png', img[y:bottom, x:right])

            try:
                send2trash(str(fname))
            except: pass    
        print('cleaning '+asana+' done.')
    return


In [9]:
detect_persons_and_save_crops("downdog", graph_def, x+'DATASET/TEST'+"downdog".lower()+'/')

cleaning downdog done.
